In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from Models import Model, ClassificationModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
df = pd.read_csv("IoT/IoTProcessed_Data.csv")
#Removing redundant columns
redundant_columns = ['Fan_actuator_OFF', 'Watering_plant_pump_OFF', 'Water_pump_actuator_OFF']
cleaned_df = df.drop(redundant_columns, axis=1)

#Converting columns to boolean
bool_columns = ['Fan_actuator_ON', 'Watering_plant_pump_ON', 'Water_pump_actuator_ON']
cleaned_df[bool_columns] = cleaned_df[bool_columns].astype(bool)

#Converting date column to date time data type
cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
#Extracting month and time from date and drop date column

cleaned_df.head()

,date,tempreature,humidity,water_level,N,P,K,Fan_actuator_ON,Watering_plant_pump_ON,Water_pump_actuator_ON
0,2024-02-08 06:10:00,41,63,100,255,255,255,True,False,False
1,2024-02-08 06:15:00,41,59,100,255,255,255,True,False,False
2,2024-02-08 06:20:00,41,62,100,255,255,255,True,False,False
3,2024-02-08 06:05:00,40,60,100,255,255,255,True,False,False
4,2024-02-08 06:00:00,39,61,100,255,255,255,True,False,False


In [3]:
temp_df = cleaned_df.drop(columns=['date'], axis=1)

In [4]:
targets = ['Fan_actuator_ON', 'Watering_plant_pump_ON', 'Water_pump_actuator_ON']
train, test = train_test_split(temp_df, train_size=0.75)

In [5]:
model = Model(train, targets, model=ClassificationModel, batch_size=128)

Using cpu


In [6]:
model.train(epochs=100)

Epoch 01/100 · train BCELoss = 0.2409
Epoch 11/100 · train BCELoss = 0.0122
Epoch 21/100 · train BCELoss = 0.0102
Epoch 31/100 · train BCELoss = 0.0090
Epoch 41/100 · train BCELoss = 0.0083
Epoch 51/100 · train BCELoss = 0.0077
Epoch 61/100 · train BCELoss = 0.0073
Epoch 71/100 · train BCELoss = 0.0070
Epoch 81/100 · train BCELoss = 0.0067
Epoch 91/100 · train BCELoss = 0.0064


In [7]:
x_test_df = test.drop(columns=targets, axis=1)
y_test_tensor = torch.from_numpy(test[targets].values.astype(np.float32))
model.eval(x_test_df, y_test_tensor, nn.BCELoss())

BCELoss: 0.0061
Accuracy: 0.9977
